In [10]:
import os
from dotenv import load_dotenv
load_dotenv(dotenv_path="workshop.env")


from pprint import pprint
from pathlib import Path
import shutil
from time import time
from pydub import AudioSegment

from core.download_webpage_as_pdf import download_webpage_as_pdf_pdfkit
from core.pdf_text_extractor import extract_text
from core.podcast_generator import PodcastGenerator
from core.salutespeech_utils import synthesize_sber_speech_sync

def write_bytes(bytes_arr, fp):
    with open(file=fp, mode="wb") as f:
        f.write(bytes_arr)

# Как сгенерировать подкаст на основе отечественных технологий

## План

1) Скачаем и сконвертируем веб страничку

2) Сгенерируем сценарий подкаста

3) Озвучим подкаст

4) А сколько это стоит?

# Скачаем страничку и извлечем текст

### pdfkit, mupdf

In [11]:
URL = "https://habr.com/ru/companies/sberbank/articles/919884/"
pdf_path = "./919884.pdf"

download_webpage_as_pdf_pdfkit(
    URL,
    pdf_path,
    ".",
    True
)
assert os.path.exists(pdf_path)


text = extract_text(pdf_path)
print(f"{len(text)} symbols, {len(text.split())} words")

Converting https://habr.com/ru/companies/sberbank/articles/919884/ to PDF: 919884.pdf
Successfully created PDF: 919884.pdf
15741 symbols, 2193 words


# Сгенерируем сценарий подкаста

### Gigachat, langchain

In [12]:
generator = PodcastGenerator(
    os.environ["GIGACHAT_USER"],
    os.environ["GIGACHAT_PASSWORD"],
    os.environ["GIGACHAT_URL"]
)

In [ ]:
tb = time()

script = generator.generate_podcast(text)
dialog = []
for line in script.split("\n"):
    line = line.strip()
    if len(line) > 0:
        replic = {}
        replic["speaker"] = line[:line.find(":")]
        replic["text"] = line[len(replic["speaker"]) + 1:]
        dialog.append(replic)


tt = time()-tb
symbols_for_tts = len(" ".join([d["text"] for d in dialog]))
print(f"Время на генерацию подкаста: {tt:.2f} sec")
print(f"Сгенерировано реплик: {len(dialog)}")
print(f"Длина текста для озвучивания: {symbols_for_tts}")

In [14]:
pprint(dialog[:4])

[{'speaker': 'Алиса',
  'text': ' Привет, Михаил! Сегодня поговорим про технологии будущего. Ты '
          'занимаешься разработкой системы, способной понимать жестовый язык '
          'глухонемых людей. Расскажи попроще, зачем это нужно?'},
 {'speaker': 'Михаил',
  'text': ' Привет, Алиса! Представь себе ситуацию: ты приходишь в кафе, '
          'заказываешь кофе, а бариста вдруг показывает тебе жестами меню. '
          'Удобно? Вот именно такую коммуникацию мы хотим сделать доступной '
          'каждому!'},
 {'speaker': 'Алиса',
  'text': ' Круто звучит! Значит, компьютер теперь сможет читать мысли через '
          'руки? Шутка... Но правда ли, что жестовый язык сложнее устного?'},
 {'speaker': 'Михаил',
  'text': ' Ха, ну считай, что наши компьютеры скоро станут телепатами! Только '
          'вот жестовый язык действительно устроен иначе. Например, порядок '
          'жестов в предложении отличается от привычного нам порядка слов. И '
          'ещё одна сложность — похожие 

## Озвучим подкаст

In [15]:
# mood = None
mood = 'whisper'
# mood = 'sad'
# mood = 'annoyed'
# mood = 'happy'

tmp_path = Path("./podcast_tmp_out")
shutil.rmtree(tmp_path, ignore_errors=True)
os.makedirs(tmp_path)


tb = time()

generated_files_in_order = []
for i, replica in enumerate(dialog):
    file_path = tmp_path/f"record_{i}.wav"
    generated_files_in_order.append(file_path)
    
    voice = "Erm_24000"
    if replica["speaker"] != "Алиса":
        voice = "Pon_24000"
    
    synthesize_sber_speech_sync(
        text=replica["text"],
        output_filename=file_path,
        format="wav16",
        voice=voice,
        mood=mood
    )

tt = time()-tb
print(f"Время на генерацию звука: {tt:.2f} sec")

2025-06-25 09:50:57.488 | INFO     | core.salutespeech_utils:synthesize_sber_speech_sync:106 - Аудио синтезировано: podcast_tmp_out/record_0.wav
2025-06-25 09:50:58.462 | INFO     | core.salutespeech_utils:synthesize_sber_speech_sync:106 - Аудио синтезировано: podcast_tmp_out/record_1.wav
2025-06-25 09:50:59.273 | INFO     | core.salutespeech_utils:synthesize_sber_speech_sync:106 - Аудио синтезировано: podcast_tmp_out/record_2.wav
2025-06-25 09:51:00.433 | INFO     | core.salutespeech_utils:synthesize_sber_speech_sync:106 - Аудио синтезировано: podcast_tmp_out/record_3.wav
2025-06-25 09:51:01.146 | INFO     | core.salutespeech_utils:synthesize_sber_speech_sync:106 - Аудио синтезировано: podcast_tmp_out/record_4.wav
2025-06-25 09:51:02.133 | INFO     | core.salutespeech_utils:synthesize_sber_speech_sync:106 - Аудио синтезировано: podcast_tmp_out/record_5.wav
2025-06-25 09:51:02.935 | INFO     | core.salutespeech_utils:synthesize_sber_speech_sync:106 - Аудио синтезировано: podcast_tmp_ou

Время на генерацию звука: 10.81 sec


Объединим кусочки

In [16]:
tb = time()

audio_segments = []
for wav_fp in generated_files_in_order:
    audio_segment = AudioSegment.from_file(wav_fp)
    audio_segments.append(audio_segment)

combined_audio = sum(audio_segments)
audio_bytes = combined_audio.export(format="mp3").read()
write_bytes(audio_bytes, tmp_path/"podcast_full.mp3")

tt = time()-tb
print(f"Время на объединение кусочков: {tt:.2f} sec")

generated_duration = int(len(combined_audio)/1000)
print(f"Сгенерировано {generated_duration} секунд подкаста")

Время на объединение кусочков: 2.07 sec
Сгенерировано 166 секунд подкаста


In [17]:
from IPython.display import display, HTML
def show_audio_with_controls(file_path):
    display(HTML("<audio controls><source src={} type='audio/mpeg'></audio>".format(file_path)))

show_audio_with_controls("./podcast_tmp_out/podcast_full.mp3")

gigachat: https://developers.sber.ru/docs/ru/gigachat/api/tariffs

salutespeech: https://developers.sber.ru/portal/products/smartspeech

In [18]:
gigachat_vect_k = 0.00004 # rub/token
gigachat_gen_k = 0.00195 # rub/token
token_usage = generator.token_callback.token_usage
gigachat_costs = token_usage["prompt_tokens"]*gigachat_vect_k + token_usage["completion_tokens"]*gigachat_gen_k
print(f"GigaChat costs: {gigachat_costs:.2f} руб.")

tts_k = 0.000186 # rub/symbol
tts_costs = symbols_for_tts * tts_k
print(f"Salutespeech costs: {tts_costs:.2f} руб.")

print(f"Total costs: {gigachat_costs+tts_costs:.2f} руб.")

GigaChat costs: 1.14 руб.
Salutespeech costs: 0.43 руб.
Total costs: 1.57 руб.
